In [17]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import copy
import seaborn as sns

import pyproj
from scipy.interpolate import griddata
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform
import geopandas as gpd

In [106]:
# Clip all images to the desired aoi.

def clip_to_aoi(path_to_aoi, img_path_list):
    '''This function clips images to an area of interest.
    Inputs:
    path_to_aoi: the absolute path to a geojson file for the aoi - this needs to be in EPSG 4326
    img_path_list: list of absolute path names for the images to clip.
    
    Output: saves the clipped images as .tif files to the same directory where the original images are stored'''
    
    with open(path_to_aoi, 'r') as f:
        aoi = json.load(f)
    aoi_geom = aoi['features'][0]['geometry']
    
    # find the coordinate reference system of the image
    for i in img_path_list:
        with rio.open(i) as src:
            dst_crs = src.crs
            wgs84 = pyproj.CRS('EPSG:4326')
            project = pyproj.Transformer.from_crs(wgs84, dst_crs, always_xy=True).transform
            proj_aoi = transform(project, shape(aoi_geom))
            
            mask, mask_transform = rio.mask.mask(src, [proj_aoi], crop=True)
            mask_meta = src.meta
            mask_meta.update({"driver": "GTiff",
                 "height": mask.shape[1],
                 "width": mask.shape[2],
                 "transform": mask_transform})
        
        with rio.open(str(os.path.abspath(i).split('.tif')[0])+"_clipped.tif", "w", **mask_meta) as dest:
            dest.write(mask)   

In [9]:
path_to_aoi = '../data/chardonnay_2020.geojson'

In [10]:
ss_20 = glob.glob('../data/images/2020/SkySat/C*/*corg_cor.tif')
ss_20

['../data/images/2020/SkySat/Coregistered_Scenes/20200812_153924_ssc3d2_0013_analytic_SR_corg_cor.tif',
 '../data/images/2020/SkySat/Coregistered_Scenes/20200805_154044_ssc13d2_0013_analytic_SR_corg_cor.tif',
 '../data/images/2020/SkySat/Coregistered_Scenes/20200618_160058_ssc3d2_0013_analytic_SR_corg_cor.tif',
 '../data/images/2020/SkySat/Coregistered_Scenes/20200710_155028_ssc3d2_0012_analytic_SR_corg_cor.tif',
 '../data/images/2020/SkySat/Coregistered_Scenes/20200625_154704_ssc12d2_0013_analytic_SR_corg_cor.tif']

In [11]:
#clip_to_aoi(path_to_aoi, ss_20)

In [13]:
ps_20 = glob.glob('../data/images/2020/PScope/2*.tif')
ps_20
#clip_to_aoi(path_to_aoi, ps_20)

In [22]:
# 2021 - use geojson for all 20 rows

path_20row = '../data/chardonnay_for_dora_EPSG4326.geojson'
ss_21 = glob.glob('../data/images/2021/SkySat/2*.tif')
ss_21
#clip_to_aoi(path_20row, ss_21)

In [24]:
ps_21 = glob.glob('../data/images/2021/PScope/2*.tif')
ps_21
#clip_to_aoi(path_20row, ps_21)

In [28]:
# 2022 - use geojson for all 20 rows

ss_22 = glob.glob('../data/images/2022/SkySat/coreg/*.tif')
ss_22
#clip_to_aoi(path_20row, ss_22)

In [34]:
ps_22 = glob.glob('../data/images/2022/PScope/2*_3B_AnalyticMS_SR_8b_*.tif')
ps_22
#clip_to_aoi(path_20row, ps_22)

In [108]:
path_20row = '../data/chardonnay_for_dora_EPSG4326.geojson'
ps_0828 = '../data/images/2022/PScope/20220828_151645_10_2262_3B_AnalyticMS_SR_8b_harmonized_clip.tif'
clip_to_aoi(path_20row, [ps_0828])

In [98]:
# Add interpolated disease bands - sev and incidence, dm and pm

def get_centroids(scout_fp, centroids_fp):
    # Drop panel geometries so gpd doesn't get confused
    scout_data = pd.read_csv(scout_fp)
    scout_data_points = scout_data.drop(['geometry'], axis=1)
    # Convert to gdf
    scout_data_points = gpd.GeoDataFrame(data=scout_data_points)
    
    centroids = gpd.read_file(centroids_fp)
    
    to_concat = centroids[['Row','Panel','xcoord','ycoord']]
    #to_concat = to_concat.rename(columns={'row':'Row', 'panel':'Panel'})
    to_concat['Row'] = pd.to_numeric(to_concat['Row'])
    to_concat['Panel']=pd.to_numeric(to_concat['Panel'])
    
    concatted = scout_data_points.merge(to_concat, on=['Row','Panel'])
    
    return concatted

def disease_grid(scout_gdf, epsg_crs, ras_fp, res, out_dir):
    from pyproj import Transformer
    from pyproj import CRS
    
    crs = CRS.from_epsg(epsg_crs)
    
    proj = Transformer.from_crs(crs.geodetic_crs, crs)
    
    x,y = proj.transform(scout_gdf.ycoord,scout_gdf.xcoord) # yes, x and y must be reversed
    
    points = list(zip(x,y))
    dis_sev = scout_gdf.total_dis.values
    dm_sev = scout_gdf.DM_severity.values
    dm_inc = scout_gdf.DM_inc.values
#     pm_sev = scout_gdf.PM_severity.values
#     pm_inc = scout_gdf.PM_inc.values


    
    with rio.open(ras_fp) as src:
        x_min= src.bounds[0]
        x_max = src.bounds[2]
        y_min = src.bounds[1]
        y_max = src.bounds[3]
        
        ras_array = src.read()
    
    xRange = np.arange(x_min,x_max,res)
    yRange = np.arange(y_max,y_min,-res)
    
    #create arrays of x,y over the raster extension
    gridX,gridY = np.meshgrid(xRange, yRange)
    
    #interpolate over the grid
    
    #Total disease
    grid_dis_sev = griddata(points, dis_sev, (gridX,gridY), method='cubic')
    grid_dis_sev = np.reshape(grid_dis_sev, (1,grid_dis_sev.shape[0], grid_dis_sev.shape[1]))
    
    #DM
    grid_dm_sev = griddata(points, dm_sev, (gridX,gridY), method='cubic')
    grid_dm_sev = np.reshape(grid_dm_sev, (1,grid_dm_sev.shape[0], grid_dm_sev.shape[1]))
    
    grid_dm_inc = griddata(points, dm_inc, (gridX,gridY), method='cubic')
    grid_dm_inc = np.reshape(grid_dm_inc, (1,grid_dm_inc.shape[0], grid_dm_inc.shape[1]))
    
    #PM
#     grid_pm_sev = griddata(points, pm_sev, (gridX,gridY), method='cubic')
#     grid_pm_sev = np.reshape(grid_pm_sev, (1,grid_pm_sev.shape[0], grid_pm_sev.shape[1]))
    
#     grid_pm_inc = griddata(points, pm_inc, (gridX,gridY), method='cubic')
#     grid_pm_inc = np.reshape(grid_pm_inc, (1,grid_pm_inc.shape[0], grid_pm_inc.shape[1]))
    
    merged = np.vstack([ras_array, grid_dis_sev, grid_dm_sev, grid_dm_inc])#, grid_pm_sev, grid_pm_inc])
    
    from rasterio.transform import Affine
    from rasterio.crs import CRS
    
    transform = Affine.translation(gridX[0][0], gridY[0][0])*Affine.scale(res,-res)
    rasterCrs = CRS.from_epsg(32618)
    
    
    with rio.open(ras_fp) as src:
        kwargs = src.meta
        band_ct = merged.shape[0]
        kwargs.update(dtype=rio.float32, count=band_ct, crs=rasterCrs, transform=transform)
        
        with rio.open(out_dir+str(os.path.basename(ras_fp))+'_disease.tif', 'w', **kwargs) as dst:
            for b in range(merged.shape[0]):
                dst.write_band(b+1, merged[b].astype(rio.float32))


In [57]:
# Test adding interpolated dis. bands to a set of clipped images

# 2020

clipped_2020 = glob.glob('../data/images/2020/PScope/clipped*/*.tif')
clipped_2020

gdf_2020 = '../data/img_scout_dfs/2020/INC_coreg_skysat_scout_2020.csv'
cent_path = '../data/panels_xy.csv'
xy_2020 = get_centroids(gdf_2020, cent_path)
xy_2020.head()

/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_1207/2828525534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_concat['row'] = pd.to_numeric(to_concat['row'])
/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_1207/2828525534.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_concat['panel']=pd.to_numeric(to_concat['panel'])


,acquired,plot,blue,green,red,nir,row,panel,Date,Treatment,Block,PM_severity,DM_severity,total_dis,centroid,PM_inc,DM_inc,xcoord,ycoord
0,20200618,1,0.040044,0.077491,0.071216,0.353575,1,1,2020-06-18,5.0,PM,0.0,0.00,0.00,POINT (-77.0153084903025 42.8783140936145),0.0,0.0,-77.01530849,42.87831409
1,20200618,2,0.044693,0.081807,0.077133,0.336587,1,2,2020-06-18,9.0,PM,0.0,0.05,5.05,POINT (-77.0153965503027 42.87833067861654),0.0,5.0,-77.01539655,42.87833068
2,20200625,2,0.053707,0.071710,0.059907,0.272707,1,2,2020-06-25,9.0,PM,0.0,0.00,0.15,POINT (-77.0153965503027 42.87833067861654),0.0,0.0,-77.01539655,42.87833068
3,20200710,2,0.048067,0.066793,0.062333,0.252887,1,2,2020-07-09,9.0,PM,0.0,0.00,0.05,POINT (-77.0153965503027 42.87833067861654),0.0,0.0,-77.01539655,42.87833068
4,20200710,2,0.048067,0.066793,0.062333,0.252887,1,2,2020-07-09,9.0,PM,0.0,0.00,15.00,POINT (-77.0153965503027 42.87833067861654),NaN,NaN,-77.01539655,42.87833068


In [58]:
clipped_2020

['../data/images/2020/PScope/clipped_2020/20200801_151354_03_2212_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif',
 '../data/images/2020/PScope/clipped_2020/20200616_151605_21_2304_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif',
 '../data/images/2020/PScope/clipped_2020/20200713_151457_44_2278_3B_AnalyticMS_SR_8b_harmonized_clip_clipped.tif']

In [60]:
# Split xy_2020 into daily dfs
dates = xy_2020.acquired.unique()

daily_dfs =[]
for date in dates:
    daily_df = xy_2020[xy_2020['acquired']==date]
    daily_dfs.append(daily_df)

dates

array([20200618, 20200625, 20200710, 20200805, 20200812])

In [62]:
# Fill DM_inc and severity NAN with zero

for df in daily_dfs:
    values = {"DM_inc": 0, "DM_severity":0}
    df = df.fillna(value=values)

In [63]:
epsg_crs = 32618
out_dir = '../data/images/2020/PScope/dis_band_2020/'

# # 20200618 image
# disease_grid(daily_dfs[0], epsg_crs, clipped_2020[1], 3, out_dir)

In [64]:
# 20200710
# disease_grid(daily_dfs[2], epsg_crs, clipped_2020[2], 3, out_dir)
# #20200805
# disease_grid(daily_dfs[3], epsg_crs, clipped_2020[0], 3, out_dir)

In [67]:
# Add dis. sev. and inc. bands for 2020 SkySat

ss_clipped_2020 = glob.glob('../data/images/2020/SkySat/clipped*/*.tif')
out_dir = '../data/images/2020/SkySat/dis_band_2020/'

In [68]:
ss_clipped_2020

['../data/images/2020/SkySat/clipped_2020/20200618_160058_ssc3d2_0013_analytic_SR_corg_cor_clipped.tif',
 '../data/images/2020/SkySat/clipped_2020/20200625_154704_ssc12d2_0013_analytic_SR_corg_cor_clipped.tif',
 '../data/images/2020/SkySat/clipped_2020/20200812_153924_ssc3d2_0013_analytic_SR_corg_cor_clipped.tif',
 '../data/images/2020/SkySat/clipped_2020/20200805_154044_ssc13d2_0013_analytic_SR_corg_cor_clipped.tif',
 '../data/images/2020/SkySat/clipped_2020/20200710_155028_ssc3d2_0012_analytic_SR_corg_cor_clipped.tif']

In [69]:
dates

array([20200618, 20200625, 20200710, 20200805, 20200812])

In [70]:
# 20200618 

# disease_grid(daily_dfs[0], epsg_crs, ss_clipped_2020[0], 0.5, out_dir)

# #20200625
# disease_grid(daily_dfs[1], epsg_crs, ss_clipped_2020[1], 0.5, out_dir)

# #20200710
# disease_grid(daily_dfs[2], epsg_crs, ss_clipped_2020[4], 0.5, out_dir)

# #20200805
# disease_grid(daily_dfs[3], epsg_crs, ss_clipped_2020[3], 0.5, out_dir)

# #20200812
# disease_grid(daily_dfs[4], epsg_crs, ss_clipped_2020[2], 0.5, out_dir)


In [77]:
# Add dis. inc. and sev. to 2021 PS and SS

ps_2021 = glob.glob("../data/images/2021/PScope/clipped_2021/*.tif")
ps_2021
ss_2021 = glob.glob("../data/images/2021/SkySat/clipped_2021/*.tif")
ss_2021
gdf_2021 = '../data/img_scout_dfs/2021/INC_skysat_scout_2021.csv'
cent_path = '../data/panels_xy.csv'
xy_2021 = get_centroids(gdf_2021, cent_path)
xy_2021.info()
out_dir_ps = '../data/images/2021/PScope/dis_band_2021/'
out_dir_ss = '../data/images/2021/SkySat/dis_band_2021/'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195 entries, 0 to 1194
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acquired     1195 non-null   int64  
 1   plot         1195 non-null   int64  
 2   blue         1195 non-null   float64
 3   green        1195 non-null   float64
 4   red          1195 non-null   float64
 5   nir          1195 non-null   float64
 6   row          1195 non-null   int64  
 7   panel        1195 non-null   int64  
 8   Date         1195 non-null   object 
 9   Treatment    1195 non-null   float64
 10  Block        1195 non-null   object 
 11  PM_severity  1195 non-null   float64
 12  DM_severity  1195 non-null   float64
 13  total_dis    1195 non-null   float64
 14  centroid     1195 non-null   object 
 15  PM_inc       1195 non-null   float64
 16  DM_inc       1195 non-null   float64
 17  xcoord       1195 non-null   object 
 18  ycoord       1195 non-null   object 
dtypes: flo

/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_1207/2828525534.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_concat['row'] = pd.to_numeric(to_concat['row'])
/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_1207/2828525534.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_concat['panel']=pd.to_numeric(to_concat['panel'])


In [78]:
# Split xy_2021 into daily dfs
dates_2021 = xy_2021.acquired.unique()

daily_dfs_2021 =[]
for date in dates_2021:
    daily_df = xy_2021[xy_2021['acquired']==date]
    daily_dfs_2021.append(daily_df)

dates_2021

array([20210707, 20210726, 20210802, 20210809, 20210816])

In [79]:
# Fill DM_inc and severity NAN with zero

for df in daily_dfs_2021:
    values = {"DM_inc": 0, "DM_severity":0}
    df = df.fillna(value=values)

In [82]:
ss_2021

['../data/images/2021/SkySat/clipped_2021/20210809_185329_ssc9d2_0014_ar_clipped.tif',
 '../data/images/2021/SkySat/clipped_2021/20210816_171007_ssc18d2_0015_ar_clipped.tif',
 '../data/images/2021/SkySat/clipped_2021/20210707_150122_ssc19d2_0016_ar_clipped.tif',
 '../data/images/2021/SkySat/clipped_2021/20210726_160338_ssc1d2_0013_ar_clipped.tif',
 '../data/images/2021/SkySat/clipped_2021/20210802_184019_ssc10d2_0005_ar_clipped.tif']

In [81]:
#PS
# 20210707 
# disease_grid(daily_dfs_2021[0], epsg_crs, ps_2021[0], 3, out_dir_ps)

# # 20210726
# disease_grid(daily_dfs_2021[1], epsg_crs, ps_2021[4], 3, out_dir_ps)

# # 20210802
# disease_grid(daily_dfs_2021[2], epsg_crs, ps_2021[1], 3, out_dir_ps)

# #20210809
# disease_grid(daily_dfs_2021[3], epsg_crs, ps_2021[2], 3, out_dir_ps)



In [83]:
#SS
# 20210707 
# disease_grid(daily_dfs_2021[0], epsg_crs, ss_2021[2], 0.5, out_dir_ss)

# # # 20210726
# disease_grid(daily_dfs_2021[1], epsg_crs, ss_2021[3], 0.5, out_dir_ss)

# # # 20210802
# disease_grid(daily_dfs_2021[2], epsg_crs, ss_2021[4], 0.5, out_dir_ss)

# # #20210809
# disease_grid(daily_dfs_2021[3], epsg_crs, ss_2021[0], 0.5, out_dir_ss)

# #20210816
# disease_grid(daily_dfs_2021[4], epsg_crs, ss_2021[1], 0.5, out_dir_ss)

In [109]:
# Add dis. inc. and sev. to 2022 PS and SS

ps_2022 = glob.glob("../data/images/2022/PScope/clipped_2022/*.tif")
ps_2022
ss_2022 = glob.glob("../data/images/2022/SkySat/clipped_2022/*.tif")
ss_2022
gdf_2022 = '../data/scout/scout_incidence/scout_inc_2022.csv'
cent_path = '../data/panels_xy.csv'
xy_2022 = get_centroids(gdf_2022, cent_path)
xy_2022.info()
out_dir_ps = '../data/images/2022/PScope/dis_band_2022/'
out_dir_ss = '../data/images/2022/SkySat/dis_band_2022/'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1871 entries, 0 to 1870
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         1871 non-null   object 
 1   Row          1871 non-null   int64  
 2   Panel        1871 non-null   int64  
 3   Treatment    1871 non-null   int64  
 4   Block        1871 non-null   object 
 5   PM_severity  1679 non-null   float64
 6   DM_severity  1871 non-null   float64
 7   total_dis    1679 non-null   float64
 8   centroid     1871 non-null   object 
 9   DM_inc       1871 non-null   float64
 10  PM_inc       1679 non-null   float64
 11  xcoord       1871 non-null   object 
 12  ycoord       1871 non-null   object 
dtypes: float64(5), int64(3), object(5)
memory usage: 204.6+ KB


/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_1207/2850047741.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_concat['Row'] = pd.to_numeric(to_concat['Row'])
/var/folders/hg/02pmwyy14wvgdv7n1gzwlx1w0000gn/T/ipykernel_1207/2850047741.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_concat['Panel']=pd.to_numeric(to_concat['Panel'])


In [110]:
# Split xy_2022 into daily dfs
dates_2022 = xy_2022.Date.unique()

daily_dfs_2022 =[]
for date in dates_2022:
    daily_df = xy_2022[xy_2022['Date']==date]
    daily_dfs_2022.append(daily_df)

dates_2022

array(['2022-06-22', '2022-06-29', '2022-07-06', '2022-07-14',
       '2022-07-20', '2022-07-27', '2022-08-02', '2022-08-09',
       '2022-08-29'], dtype=object)

In [111]:
# Fill DM_inc and severity NAN with zero

for df in daily_dfs_2022:
    values = {"DM_inc": 0, "DM_severity":0}
    df = df.fillna(value=values)

In [114]:
ss_2022

['../data/images/2022/SkySat/clipped_2022/20220801_151508_ssc13d2_0011_analytic_SR_corg_clipped.tif',
 '../data/images/2022/SkySat/clipped_2022/20220622_160829_ssc15d2_0014_analytic_SR_corg_clipped.tif',
 '../data/images/2022/SkySat/clipped_2022/20220720_153112_ssc4d2_0013_analytic_SR_clipped_clipped.tif']

In [113]:
#PS
# 2022-06-22 
# disease_grid(daily_dfs_2022[0], epsg_crs, ps_2022[5], 3, out_dir_ps)

# # 2022-06-29
# disease_grid(daily_dfs_2022[1], epsg_crs, ps_2022[0], 3, out_dir_ps)

# # 2022-07-06
# disease_grid(daily_dfs_2022[2], epsg_crs, ps_2022[8], 3, out_dir_ps)

# #2022-07-27
# disease_grid(daily_dfs_2022[5], epsg_crs, ps_2022[9], 3, out_dir_ps)

# #2022-08-02
# disease_grid(daily_dfs_2022[6], epsg_crs, ps_2022[11], 3, out_dir_ps)

# #2022-08-29
# disease_grid(daily_dfs_2022[8], epsg_crs, ps_2022[3], 3, out_dir_ps)


In [115]:
#SS
# # 20220622 
# disease_grid(daily_dfs_2022[0], epsg_crs, ss_2022[1], 0.5, out_dir_ss)

# # 20220720
# disease_grid(daily_dfs_2022[4], epsg_crs, ss_2022[2], 0.5, out_dir_ss)

# # 20220801
# disease_grid(daily_dfs_2022[6], epsg_crs, ss_2022[0], 0.5, out_dir_ss)

In [21]:
# Add smr band
endmember_file = '../data/endmembers_msi_noSwir.csv'

# Next: array to raster

def smr_weights_raster(image, em_file) :
    '''Calculates the spectral mixture residual (smr) weights for each pixel
    in an image based on input endmember file.
    
    image: (string) path to raster 
    em_file: (string) path to .csv with reflectance values for each endmember
    
    returns: array with original image bands + endmembers'''
       

    # ATTENTION, MAKE SURE ENDMEMBER FILE IS ACCESSIBLE
    
    endmembers = endmember_file
    
    with rio.open(image) as src:
        raster_arr = np.array(src.read())
        raster_arr = raster_arr/10000 #scale reflectance values
   

    w = 1 # sum constraint weight
    D = raster_arr # numpy-nD array
    d = np.reshape(D, [D.shape[0], D.shape[1]*D.shape[2]]) # dimensions
    G = pd.read_csv(endmembers, sep='[,|\t]', header = None).to_numpy()
    
    wavelength = G[:,0]
    G = G[:, 1:G.shape[1]].astype(np.float64)

    d_constraint = np.array(w*np.ones(d.shape[1]))
    G_constraint = np.array(w*np.ones(G.shape[1]))

    d = np.vstack([d, d_constraint])
    G = np.vstack([G, G_constraint])

    mixtureResidual_alt = d-(G.dot(np.linalg.inv(G.transpose().dot(G)).dot(G.transpose()))).dot(d)
    mixtureResidual_alt_reShp = np.reshape(mixtureResidual_alt,[d.shape[0],D.shape[1],D.shape[2]])
    
    M = np.linalg.inv(G.transpose().dot(G)).dot(G.transpose().dot(d))
    M_reShp = np.reshape(M,[M.shape[0], D.shape[1], D.shape[2]])
    
    w1 = M_reShp[0]
    w2 = M_reShp[1]
    
    em_ratio = np.divide(w1,\
                         w1+w2,\
                         out=np.zeros_like(w1),\
                         where=(w1+w2)!=0)
    
    #Reshape em_ratio to enable stacking with original 3D img array
    
    em_ratio_rs = em_ratio.reshape(1,em_ratio.shape[0],em_ratio.shape[1])
    
    stacked_arr = np.vstack([raster_arr, em_ratio_rs])
    
    # Save stacked array as raster
    with rio.open(image) as src:
        kwargs = src.meta
        band_ct = stacked_arr.shape[0]
        kwargs.update(dtype=rio.float32, count=band_ct)
        
        with rio.open(str(os.path.split(image)[0])+'/SMR_'+str(os.path.basename(image)), 'w', **kwargs) as dst:
            for b in range(stacked_arr.shape[0]):
                dst.write_band(b+1, stacked_arr[b].astype(rio.float32))